# SKU-110K Dataset Processing Notebook

## 2.1 Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

## 2.2 Verify Dataset Structure

In [ ]:
!ls /content/drive/MyDrive

## 2.3 Extract Dataset

In [ ]:
import tarfile
src_file = '/content/drive/MyDrive/SKU110K_fixed.tar.gz'
dest_path = '/content/drive/MyDrive/datasets/SKU110K_data'

with tarfile.open(src_file, "r:gz") as tar:
    tar.extractall(path=dest_path)

## 2.4 Check Directory Structure

In [ ]:
import os
base_path = "/content/drive/MyDrive/datasets/SKU110K_data/SKU110K_fixed"

for root, dirs, files in os.walk(base_path):
    print(f"📂 Directory: {root}")
    print(f"📦 Subdirectories: {dirs}")
    print(f"📄 Files: {files}")
    break  # Show first level only

## 3. Data Validation & Cleaning

### 3.1 Check for Corrupted Images

In [ ]:
from PIL import Image
image_dir = os.path.join(base_path, 'images')
bad_images = []

for img_name in os.listdir(image_dir)[:500]:  # Sample check
    img_path = os.path.join(image_dir, img_name)
    try:
        img = Image.open(img_path)
        img.verify()
    except Exception as e:
        bad_images.append((img_name, str(e)))

print(f"❌ Found {len(bad_images)} corrupted images.")

### 3.2 Load & Validate Annotations

In [ ]:
import pandas as pd
ann_path = os.path.join(base_path, "annotations/annotations_test.csv")

# Load with correct column names
df = pd.read_csv(ann_path, names=[
    "image_name", "x1", "y1", "x2", "y2", "class", "image_width", "image_height"
])

# Check structure
df.head()

### 3.3 Data Quality Checks

In [ ]:
# Drop duplicates
df = df.drop_duplicates().reset_index(drop=True)

# Convert to numeric
numeric_cols = ["x1", "y1", "x2", "y2", "image_width", "image_height"]
df[numeric_cols] = df[numeric_cols].apply(pd.to_numeric)

# Check for invalid bounding boxes
invalid_coords = df[(df.x1 >= df.x2) | (df.y1 >= df.y2)]
print(f"❌ Found {len(invalid_coords)} invalid boxes.")

# Check for out-of-bounds coordinates
out_of_bounds = df[
    (df.x2 > df.image_width) | (df.y2 > df.image_height) |
    (df.x1 < 0) | (df.y1 < 0)
]
print(f"❌ Found {len(out_of_bounds)} out-of-bounds boxes.")

## 4. COCO Format Conversion

### 4.1 Convert Annotations to COCO Format

In [ ]:
import json
from tqdm import tqdm

def convert_to_coco(df):
    images = []
    annotations = []
    categories = [{"id": 1, "name": "object"}]

    image_id_map = {}  # Maps image_name → image_id
    annotation_id = 1

    for idx, row in tqdm(df.iterrows(), total=len(df)):
        img_name = row["image_name"]
        if img_name not in image_id_map:
            image_id = len(image_id_map) + 1
            image_id_map[img_name] = image_id
            images.append({
                "id": image_id,
                "file_name": img_name,
                "width": row["image_width"],
                "height": row["image_height"]
            })

        x, y = row["x1"], row["y1"]
        width, height = row["x2"] - x, row["y2"] - y

        annotations.append({
            "id": annotation_id,
            "image_id": image_id_map[img_name],
            "category_id": 1,
            "bbox": [x, y, width, height],
            "area": width * height,
            "iscrowd": 0
        })
        annotation_id += 1

    return {"images": images, "annotations": annotations, "categories": categories}

# Convert and save
coco = convert_to_coco(df)
output_json = "/content/drive/MyDrive/datasets/SKU110K_project/processed/annotations_coco.json"
os.makedirs(os.path.dirname(output_json), exist_ok=True)

with open(output_json, "w") as f:
    json.dump(coco, f)

print(f"✅ COCO annotations saved to: {output_json}")

## 5. Metadata Generation

### 5.1 Create Dataset Profile

In [ ]:
from datetime import datetime

dataset_profile = {
    "dataset_name": "SKU-110K",
    "version": "v1.0",
    "num_images": len(coco['images']),
    "num_annotations": len(coco['annotations']),
    "class_names": [cat['name'] for cat in coco['categories']],
    "raw_images_path": "/datasets/SKU110K_project/raw/images/",
    "annotations_path": output_json,
    "created_at": datetime.now().isoformat()
}

# Save metadata
metadata_output_path = "/content/drive/MyDrive/datasets/SKU110K_project/metadata/dataset_profile.json"
os.makedirs(os.path.dirname(metadata_output_path), exist_ok=True)

with open(metadata_output_path, "w") as f:
    json.dump(dataset_profile, f, indent=4)

print("📊 Dataset metadata saved successfully.")

## 6. Planogram Data Processing (Mock Example)

### 6.1 Generate Mock Planogram Data

In [ ]:
mock_planogram = [
    {"store_id": "store_01", "product_id": "SKU_001", "x": 100, "y": 200, "width": 50, "height": 120, "shelf": "top"},
    {"store_id": "store_01", "product_id": "SKU_002", "x": 180, "y": 210, "width": 60, "height": 110, "shelf": "middle"},
    {"store_id": "store_01", "product_id": "SKU_003", "x": 260, "y": 220, "width": 55, "height": 115, "shelf": "bottom"}
]

# Save as JSON
planogram_path = "/content/drive/MyDrive/datasets/SKU110K_project/raw/planogram.json"
os.makedirs(os.path.dirname(planogram_path), exist_ok=True)

with open(planogram_path, "w") as f:
    json.dump(mock_planogram, f, indent=4)

print(f"✅ Mock planogram saved to: {planogram_path}")

### 6.2 Convert to CSV & SQLite

In [ ]:
# Load and validate
with open(planogram_path, "r") as f:
    data = json.load(f)

df_planogram = pd.DataFrame(data)

# Save as CSV
csv_output_path = "/content/drive/MyDrive/datasets/SKU110K_project/processed/planogram_clean.csv"
df_planogram.to_csv(csv_output_path, index=False)

# Save to SQLite
import sqlite3
db_path = "/content/drive/MyDrive/datasets/SKU110K_project/processed/planogram.db"
conn = sqlite3.connect(db_path)
df_planogram.to_sql("planogram", conn, if_exists="replace", index=False)
conn.close()

print("✅ Planogram processing complete.")